In [2]:
import numpy as np
from collections import Counter
import time


def load_csv(data_file_path, class_index=-1):
    """Load csv data in a numpy array.
    Args:
        data_file_path (str): path to data file.
        class_index (int): slice output by index.
    Returns:
        features, classes as numpy arrays if class_index is specified,
            otherwise all as nump array.
    """

    handle = open(data_file_path, 'r')
    contents = handle.read()
    handle.close()
    rows = contents.split('\n')
    out = np.array([[float(i) for i in r.split(',')] for r in rows if r])

    if(class_index == -1):
        classes= out[:,class_index]
        features = out[:,:class_index]
        return features, classes
    elif(class_index == 0):
        classes= out[:, class_index]
        features = out[:, 1:]
        return features, classes

    else:
        return out
    
    
class DecisionNode:
    """Class to represent a single node in a decision tree."""

    def __init__(self, left, right, decision_function, class_label=None):
        """Create a decision function to select between left and right nodes.
        Note: In this representation 'True' values for a decision take us to
        the left. This is arbitrary but is important for this assignment.
        Args:
            left (DecisionNode): left child node.
            right (DecisionNode): right child node.
            decision_function (func): function to decide left or right node.
            class_label (int): label for leaf node. Default is None.
        """

        self.left = left
        self.right = right
        self.decision_function = decision_function
        self.class_label = class_label

    def _get_right(self):
        return(self.right)
    
    def _get_left(self):
        return(self.right)
    
    def decide(self, feature):
        # """Get a child node based on the decision function.͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        # Args:͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        #     feature (list(int)): vector for feature.͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        # Return:͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        #     Class label if a leaf node, otherwise a child node.͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        # """͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
        """Determine recursively the class of an input array by testing a value
           against a feature's attributes values based on the decision function.

        Args:
            feature: (numpy array(value)): input vector for sample.

        Returns:
            Class label if a leaf node, otherwise a child node.
        """

        if self.class_label is not None:
            return self.class_label

        elif self.decision_function(feature):
            # print('LEFT <----')
            return self.left.decide(feature)

        else:
            # print('----> Right')

            return self.right.decide(feature)


In [3]:
path = './data/vectorize.csv'
path23 = './data/part23_data.csv'
path_chal = './data/challenge_train.csv'

class_t = True
data = load_csv(path_chal)
if not class_t:
    data = data[0]

In [4]:
data

(array([[ 0.,  1.,  2., ..., 27., 28., 29.],
        [ 0.,  2.,  3., ..., 37., 38., 39.],
        [ 0.,  3.,  4., ..., 41., 42., 43.],
        ...,
        [ 0.,  3.,  4., ..., 54., 59., 65.],
        [ 0.,  3.,  4., ..., 50., 51., 54.],
        [ 0.,  1.,  3., ..., 51., 54., 59.]]),
 array([30., 40., 44., ..., 68., 59., 65.]))

# Vectorized

In [88]:


class Vectorization:
    """Vectorization preparation for Assignment 5."""

    def __init__(self):
        pass

    def non_vectorized_loops(self, data):
        """Element wise array arithmetic with loops.
        This function takes one matrix, multiplies by itself and then adds to
        itself.
        Args:
            data: data to be added to array.
        Returns:
            Numpy array of data.
        """

        non_vectorized = np.zeros(data.shape)
        for row in range(data.shape[0]):
            for col in range(data.shape[1]):
                non_vectorized[row][col] = (data[row][col] * data[row][col] +
                                            data[row][col])
        return non_vectorized

    def vectorized_loops(self, data):
        """Element wise array arithmetic using vectorization.
        This function takes one matrix, multiplies by itself and then adds to
        itself.
        Args:
            data: data to be sliced and summed.
        Returns:
            Numpy array of data.
        """

        return (data[:][:]*data[:][:]) + data[:][:]

    def non_vectorized_slice(self, data):
        """Find row with max sum using loops.
        This function searches through the first 100 rows, looking for the row
        with the max sum. (ie, add all the values in that row together).
        Args:
            data: data to be added to array.
        Returns:
            Tuple (Max row sum, index of row with max sum)
        """

        max_sum = 0
        max_sum_index = 0
        for row in range(100):
            temp_sum = 0
            for col in range(data.shape[1]):
                temp_sum += data[row][col]

            if temp_sum > max_sum:
                max_sum = temp_sum
                max_sum_index = row

        return max_sum, max_sum_index

    def vectorized_slice(self, data):
        """Find row with max sum using vectorization.
        This function searches through the first 100 rows, looking for the row
        with the max sum. (ie, add all the values in that row together).
        Args:
            data: data to be sliced and summed.
        Returns:
            Tuple (Max row sum, index of row with max sum)
        """
        sums = np.sum(data[:100][:], axis=1)
        indx = np.where(sums == max(sums))[0][0]
        max_sum = sums[indx]
        return (max_sum, indx)

    def non_vectorized_flatten(self, data):
        """Display occurrences of positive numbers using loops.
         Flattens down data into a 1d array, then creates a dictionary of how
         often a positive number appears in the data and displays that value.
         ie, [(1203,3)] = integer 1203 appeared 3 times in data.
         Args:
            data: data to be added to array.
        Returns:
            List of occurrences [(integer, number of occurrences), ...]
        """

        unique_dict = {}
        flattened = np.hstack(data)
        for item in range(len(flattened)):
            if flattened[item] > 0:
                if flattened[item] in unique_dict:
                    unique_dict[flattened[item]] += 1
                else:
                    unique_dict[flattened[item]] = 1

        return unique_dict.items()

    def vectorized_flatten(self, data):
        """Display occurrences of positive numbers using vectorization.
         Flattens down data into a 1d array, then creates a dictionary of how
         often a positive number appears in the data and displays that value.
         ie, [(1203,3)] = integer 1203 appeared 3 times in data.
         Args:
            data: data to be added to array.
        Returns:
            List of occurrences [(integer, number of occurrences), ...]
        """

        return(list(Counter(np.hstack(data)[np.hstack(data) > 0]).items()))

    
    
    def non_vectorized_glue(self, data, vector, dimension='c'):
        """Element wise array arithmetic with loops.
        This function takes a multi-dimensional array and a vector, and then combines
        both of them into a new multi-dimensional array. It must be capable of handling
        both column and row-wise additions.
        Args:
            data: multi-dimensional array.
            vector: either column or row vector
            dimension: either c for column or r for row
        Returns:
            Numpy array of data.
        """
        if dimension == 'c' and len(vector) == data.shape[0]:
            non_vectorized = np.ones((data.shape[0],data.shape[1]+1), dtype=float)
            non_vectorized[:, -1] *= vector
        elif dimension == 'r' and len(vector) == data.shape[1]:
            non_vectorized = np.ones((data.shape[0]+1,data.shape[1]), dtype=float)
            non_vectorized[-1, :] *= vector
        else:
            raise ValueError('This parameter must be either c for column or r for row')
        for row in range(data.shape[0]):
            for col in range(data.shape[1]):
                non_vectorized[row, col] = data[row, col]
        return non_vectorized

    def vectorized_glue(self, data, vector, dimension='c'):
        """Array arithmetic without loops.
        This function takes a multi-dimensional array and a vector, and then combines
        both of them into a new multi-dimensional array. It must be capable of handling
        both column and row-wise additions.
        Args:
            data: multi-dimensional array.
            vector: either column or row vector
            dimension: either c for column or r for row
        Returns:
            Numpy array of data.
            
        """
        if dimension == 'c' and len(vector) == data.shape[0]:
            new_vec = vector.reshape(data.shape[0],1)
            data = np.append(data, new_vec, axis= 1)
        elif dimension == 'r' and len(vector) == data.shape[1]:
            new_vec = vector.reshape(1,data.shape[1])
            data = np.append(data, new_vec, axis= 0)
        else:
            raise ValueError('This parameter must be either c for column or r for row')
        return data
    
    def non_vectorized_mask(self, data, threshold):
        """Element wise array evaluation with loops.
        This function takes a multi-dimensional array and then populates a new
        multi-dimensional array. If the value in data is below threshold it
        will be squared.
        Args:
            data: multi-dimensional array.
            threshold: evaluation value for the array if a value is below it, it is squared
        Returns:
            Numpy array of data.
        """
        non_vectorized = np.zeros_like(data, dtype=float)
        for row in range(data.shape[0]):
            for col in range(data.shape[1]):
                val = data[row, col]
                if val >= threshold:
                    non_vectorized[row, col] = val
                    continue
                non_vectorized[row, col] = val**2

        return non_vectorized

    def vectorized_mask(self, data, threshold):
        """Array evaluation without loops.
        This function takes a multi-dimensional array and then populates a new
        multi-dimensional array. If the value in data is below threshold it
        will be squared. You are required to use a binary mask for this problem
        Args:
            data: multi-dimensional array.
            threshold: evaluation value for the array if a value is below it, it is squared
        Returns:
            Numpy array of data.
        """
        mask = np.ones((data.shape[0],data.shape[1]))
        mask[data < threshold] = 2
        data = np.power(data, mask)
        return data


# Information Theory

In [149]:
import pandas as pd 
feature = np.array([[1 ,   0,    0,    0,    1],  
 [1 ,   0,    1,    1,    1],  
 [0 ,   1,    0,    0,    1],  
 [0 ,   1,    1,    0,    0],  
 [1 ,   1,    0,    1,    1],  
 [0 ,   1,    0,    1,    0],  
 [0 ,   0,    1,    1,    1],  
 [0 ,   0,    1,    0,    0]])

def b (q):
    # print(q)
    if q == 1 or q == 0:
        # return - (q * np.log2(q))
        return 0
    # if q == 0: 
    #     return - (1-q) * np.log2(1-q)
    return - ((q * np.log2(q)) +  (1-q) * np.log2(1-q))

ft = pd.DataFrame(feature, columns=['A1', 'A2', 'A3', 'A4', 'Y'])
for i in ['A1', 'A2', 'A3', 'A4']:
    dp = len(ft[i])
    one = len(ft[i][ft[i] > 0 ])
    label_res_ppos = Counter(ft['Y'][ft[i] > 0 ])[1]
    zeros = dp - one
    label_res_npos = Counter(ft['Y'][ft[i] < 1 ])[1]
    score = b(5/dp) - (((zeros / dp) * b(label_res_npos / zeros)) +  ((one / dp) * b(label_res_ppos / one)))
    print(score)

0.34758988139079716
0.04879494069539858
0.04879494069539858
0.04879494069539858


In [150]:
pb = ft[ft['A1'] > 0]
zb = ft[ft['A1'] == 0]
dd = zb.drop(['A1'], axis=1)
dd[dd['A4'] == 0]

,A2,A3,A4,Y
2,1,0,0,1
3,1,1,0,0
7,0,1,0,0


In [151]:
ft = pb.drop(['A1'], axis=1)
for i in ['A2', 'A3', 'A4']:
    dp = len(ft[i])
    one = len(ft[i][ft[i] > 0 ])
    label_res_ppos = Counter(ft['Y'][ft[i] > 0 ])[1]
    zeros = dp - one
    label_res_npos = Counter(ft['Y'][ft[i] < 1 ])[1]
    score = b(3/dp) - (((zeros / dp) * b(label_res_npos / zeros)) +  ((one / dp) * b(label_res_ppos / one)))
    print(score)

0.0
0.0
0.0


In [152]:
def b (q):
    # print(q)
    if q == 1 or q == 0:
        # return - (q * np.log2(q))
        return 0
    if q == 0: 
        return - (1-q) * np.log2(1-q)
    return - ((q * np.log2(q)) +  (1-q) * np.log2(1-q))

In [153]:
b(9/14)

0.9402859586706311

In [154]:
bit_need = -(5/14 * np.log2(5/14) + 3/8 * np.log2(3/8))

In [155]:
new_shit = np.zeros((8,4))
label_crap = np.zeros((8,1)) 
for i in range(feature.shape[0]):
    new_shit[i] = feature[i][:-1]
    label_crap[i] = feature[i][-1]

In [156]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state=0)
forest.fit(new_shit, label_crap)

C:\Users\Kiavosh\AppData\Local\Temp\ipykernel_35820\1564465926.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(new_shit, label_crap)


RandomForestClassifier(random_state=0)

# Build Tree

In [6]:
a4_r = DecisionNode(None, None, None, 1)
a4_l = DecisionNode(None, None, None, 0)
a4l_r = DecisionNode(None, None, None, 0)
a4l_l = DecisionNode(None, None, None, 1)

a3_r = DecisionNode(a4_l, a4_r, lambda a4: a4[3] == 0)
a3_l = DecisionNode(a4l_l, a4l_r, lambda a4: a4[3] == 0)

root_r = DecisionNode(None, None, None, 1)
root_l = DecisionNode(a3_l, a3_r, lambda a3: a3[2] == 0)

root_a1 = DecisionNode(root_l, root_r, lambda a1: a1[0] == 0)


In [7]:
print(root_a1)

In [8]:
ht_examples = [[1, 0, 0, 0],
            [1, 0, 1, 1],
            [0, 1, 0, 0],
            [0, 1, 1, 0],
            [1, 1, 0, 1],
            [0, 1, 0, 1],
            [0, 0, 1, 1],
            [0, 0, 1, 0]]
ht_classes = [1, 1, 1, 0, 1, 0, 1, 0]


for index in range(0, len(ht_examples)):
    print(ht_examples[index])
    decision = root_a1.decide(ht_examples[index])
    print(decision)


[1, 0, 0, 0]
----> Right
1
[1, 0, 1, 1]
----> Right
1
[0, 1, 0, 0]
LEFT <----
LEFT <----
LEFT <----
1
[0, 1, 1, 0]
LEFT <----
----> Right
LEFT <----
0
[1, 1, 0, 1]
----> Right
1
[0, 1, 0, 1]
LEFT <----
LEFT <----
----> Right
0
[0, 0, 1, 1]
LEFT <----
----> Right
----> Right
1
[0, 0, 1, 0]
LEFT <----
----> Right
LEFT <----
0


In [142]:
for index in range(0, len(ht_examples)):
    tree = [
        [
            [
                [1,0]
            ],
            [
                [0,1]
            ]
        ],
        1
    ]
    print(ht_examples[index])

    for i in ht_examples[index]:
        if type(tree) == int: 
            continue
        print(i)
        print(tree)
        tree = tree[i]
    print('_____________')
    # print(tree)

[1, 0, 0, 0]
1
[[[[1, 0]], [[0, 1]]], 1]
_____________
[1, 0, 1, 1]
1
[[[[1, 0]], [[0, 1]]], 1]
_____________
[0, 1, 0, 0]
0
[[[[1, 0]], [[0, 1]]], 1]
1
[[[1, 0]], [[0, 1]]]
0
[[0, 1]]
0
[0, 1]
_____________
[0, 1, 1, 0]
0
[[[[1, 0]], [[0, 1]]], 1]
1
[[[1, 0]], [[0, 1]]]
1
[[0, 1]]


IndexError: list index out of range

In [ ]:
tree = [
    [
        [
            [1],[0]
        ],
        [
            [0],[1]
        ]
    ],
    1
]

# Metrics

In [9]:
answer = [1, 0, 0, 1, 0, 0, 0]
true_label = [1, 1, 1, 0, 0, 0, 0]

Counter(answer)[0]

5

In [19]:
a= np.array(answer)
t= np.array(true_label)
np.logical_and(a,a+t)


array([ True, False, False,  True, False, False, False])

In [21]:
matrix = np.zeros((2,2))
for i in range(len(true_label)):
    if true_label[i] == answer[i]:
        matrix[true_label[i]][true_label[i]] += 1
    else: 
        matrix[true_label[i]][answer[i]] += 1
        
[[matrix[1][1], matrix[1][0]], [matrix[0][1], matrix[0][0]]]

In [23]:
[[matrix[1][1], matrix[1][0]], [matrix[0][1], matrix[0][0]]]

[[1.0, 2.0], [1.0, 3.0]]

In [30]:
def confusion_matrix(classifier_output, true_labels):
    """Create a confusion matrix to measure classifier performance.

    Classifier output vs true labels, which is equal to:
    Predicted  vs  Actual Values.

    Output will in the format:

                        |Predicted|
    |T|                
    |R|    [[true_positive, false_negative],
    |U|    [false_positive, true_negative]]
    |E|

    Args:
        classifier_output (list(int)): output from classifier.
        true_labels: (list(int): correct classified labels.
    Returns:
        A two dimensional array representing the confusion matrix.
    """

    matrix = np.zeros((2,2))
    for i in range(len(true_labels)):
        if true_labels[i] == classifier_output[i]:
            matrix[true_labels[i]][true_labels[i]] += 1
        else: 
            matrix[true_labels[i]][classifier_output[i]] += 1
            
    return [[matrix[1][1], matrix[1][0]], [matrix[0][1], matrix[0][0]]]

def test_accuracy_calculation():
    """Test accuracy calculation.

    Asserts:
        Accuracy matches for all true labels.
    """

    answer = [0, 0, 0, 0, 0]
    true_label = [1, 1, 1, 1, 1]
    total_count = len(answer)

    for index in range(0, len(answer)):
        answer[index] = 1
        accuracyc = accuracy(answer, true_label)

        print(accuracyc)

def accuracy(classifier_output, true_labels):
    """Get the accuracy of a classifier compared to the correct values.
    Accuracy is measured as:
        correct_classifications / total_number_examples
    Args:
        classifier_output (list(int)): output from classifier.
        true_labels: (list(int): correct classified labels.
    Returns:
        The accuracy of the classifier output.
    """
    cm = confusion_matrix(classifier_output, true_labels)
    return (cm[0][0] + cm[1][1]) / len(classifier_output)

In [31]:
test_accuracy_calculation()

0.2
0.4
0.6
0.8
1.0


# GINI

In [163]:
import pandas as pd 
feature = np.array([[1 ,   0,    0,    0,    1],  
 [1 ,   0,    1,    1,    1],  
 [0 ,   1,    0,    0,    1],  
 [0 ,   1,    1,    0,    0],  
 [1 ,   1,    0,    1,    1],  
 [0 ,   1,    0,    1,    0],  
 [0 ,   0,    1,    1,    1],  
 [0 ,   0,    1,    0,    0]])

def b (q):
    # print(q)
    if q == 1 or q == 0:
        # return - (q * np.log2(q))
        return 0
    # if q == 0: 
    #     return - (1-q) * np.log2(1-q)
    return - ((q * np.log2(q)) +  (1-q) * np.log2(1-q))

ft = pd.DataFrame(feature, columns=['A1', 'A2', 'A3', 'A4', 'Y'])
for i in ['A1', 'A2', 'A3', 'A4']:
    dp = len(ft[i])
    one = len(ft[i][ft[i] > 0 ])
    label_res_ppos = Counter(ft['Y'][ft[i] > 0 ])[1]
    zeros = dp - one
    label_res_npos = Counter(ft['Y'][ft[i] < 1 ])[1]
    score = b(5/dp) - (((zeros / dp) * b(label_res_npos / zeros)) +  ((one / dp) * b(label_res_ppos / one)))
    print(score)

0.34758988139079716
0.04879494069539858
0.04879494069539858
0.04879494069539858


In [21]:
for i in range (0,4):
    false_list = feature[feature[:,i] == 0]
    true_list = feature[feature[:,i] == 1]
    

[1 1 0 0 1 0 0 0]
[0 0 1 1 1 1 0 0]
[0 1 0 1 0 0 1 1]
[0 1 0 0 1 1 1 0]


In [37]:
false_list = feature[:,4][feature[:,0] == 0]
true_list = feature[:,4][feature[:,0] == 1]
false_zo_count = Counter(false_list)
true_zo_count = Counter(true_list)
leaf_gini_imp_false = 1 - np.power(false_zo_count[0] / len(false_list) , 2) - np.power(false_zo_count[1] / len(false_list) , 2)
leaf_gini_imp_true = 1 - np.power(true_zo_count[0] / len(true_list) , 2) - np.power(true_zo_count[1] / len(true_list) , 2)

In [164]:
def gini_impurity(class_vector):
    """Compute the gini impurity for a list of classes.
    This is a measure of how often a randomly chosen element
    drawn from the class_vector would be incorrectly labeled
    if it was randomly labeled according to the distribution
    of the labels in the class_vector.
    It reaches its minimum at zero when all elements of class_vector
    belong to the same class.
    Args:
        class_vector (list(int)): Vector of classes given as 0 or 1.
    Returns:
        Floating point number representing the gini impurity.
    """
    if len(class_vector) <= 0:
        return 1
    false_zo_count = Counter(class_vector)
    leaf_gini_imp_false = 1 - np.power(false_zo_count[0] / len(class_vector) , 2) - np.power(false_zo_count[1] / len(class_vector) , 2)
    return leaf_gini_imp_false

def gini_gain(previous_classes, current_classes):
    """Compute the gini impurity gain between the previous and current classes.
    Args:
        previous_classes (list(int)): Vector of classes given as 0 or 1.
        current_classes (list(list(int): A list of lists where each list has
            0 and 1 values).
    Returns:
        Floating point number representing the information gain.
    """
    # return (gini_impurity(previous_classes) - np.average([gini_impurity(i) for i in current_classes]))
    return (gini_impurity(previous_classes) - np.sum([gini_impurity(i) * len(i) for i in current_classes]) / len(np.hstack(current_classes)))


In [104]:
c = gini_gain([1, 1, 1, 0, 0, 0],
                        [[1, 1, 1], [0, 0, 0]])

# assert .500 == round(gini_gain, 3)
c

0.5

In [105]:
s = gini_gain([1, 1, 1, 0, 0, 0],[[1, 1, 0], [1, 0, 0]])
round(s, 3)

0.056

In [106]:
gain_type = round(gini_gain(
            restaurant['restaurants'],
            restaurant['split_patrons']), 2)
gain_type

0.28

In [74]:
restaurant = {'restaurants': [0] * 6 + [1] * 6,
                           'split_patrons': [[0, 0],
                                             [1, 1, 1, 1],
                                             [1, 1, 0, 0, 0, 0]],
                           'split_food_type': [[0, 1],
                                               [0, 1],
                                               [0, 0, 1, 1],
                                               [0, 0, 1, 1]]}


In [77]:
restaurant['restaurants']

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

In [81]:
restaurant['split_patrons']

[[0, 0], [1, 1, 1, 1], [1, 1, 0, 0, 0, 0]]

In [101]:
val = np.sum([gini_impurity(i) * len(i) for i in restaurant['split_patrons']]) / len(np.hstack(restaurant['split_patrons']))

In [87]:
0.5- 0.278

0.22199999999999998

In [96]:
np.array(restaurant['split_patrons'])

c:\Users\Kiavosh\anaconda3\envs\ai_env\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([list([0, 0]), list([1, 1, 1, 1]), list([1, 1, 0, 0, 0, 0])],
      dtype=object)

In [97]:
np.hstack(restaurant['split_patrons'])

array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

# DT

In [123]:
t = pd.DataFrame(Vectorization.vectorized_glue(None ,data[0], data[1])
, columns=['A1', 'A2', 'A3', 'A4', 'Y'])

In [124]:
data[1]

array([0., 0., 0., ..., 1., 1., 1.])

In [121]:
Vectorization.vectorized_glue(None ,data[0], data[1])

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699,   0.     ],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ,   0.     ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645,   0.     ],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ,   1.     ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ,   1.     ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ,   1.     ]])

In [245]:
def __build_tree__(features, classes, depth=0):
    """Build tree that automatically finds the decision functions.
    Args:
        features (m x n): m examples with n features.
        classes (m x 1): Array of Classes.
        depth (int): depth to build tree to.
    Returns:
        Root node of decision tree.
    """

    if depth == 10 :
        y = Counter(classes).most_common()[0][0]
        return DecisionNode(None, None, None, y)
    
    if len(set(classes)) == 1 :
        y = Counter(classes).most_common()[0][0]
        return DecisionNode(None, None, None, y)
    
    # Finding the best split
    ledger = [0,0,0,None,None]
    for feature in range(features.shape[1]):
        featured_sorted = features[np.argsort(features[:, feature])]
        classes_sorted = classes[np.argsort(features[:, feature])]
        for dp in range(0, len(features) - 1):
            if featured_sorted[dp, feature] != featured_sorted[dp+1, feature]:
                curr_gini_info = gini_gain(classes_sorted, [classes_sorted[:dp+1], classes_sorted[dp+1:]])
                if curr_gini_info > ledger[0]:
                    ledger[0] = curr_gini_info
                    ledger[1] = feature
                    ledger[2] = dp
                    ledger[3] = classes_sorted
                    ledger[4] = featured_sorted

    if ledger[0] <= 0:
        return DecisionNode(None, None, None, y)

    curr_features = ledger[4]
    curr_classes = ledger[3]

    
    split_value = curr_features[ledger[2], ledger[1]]
    left_leaf = __build_tree__(curr_features[:ledger[2]+1], curr_classes[:ledger[2]+1], depth = depth + 1)
    right_leaf = __build_tree__(curr_features[ledger[2]+1:], curr_classes[ledger[2]+1:], depth = depth + 1)

    curr_root = DecisionNode(left_leaf, right_leaf, lambda feat: feat[ledger[1]] < split_value)

    return curr_root

In [229]:
depth = 10
# Finding the best split
sort_tracker = []


for feature in range(features.shape[1]):

        featured_sorted = features[np.argsort(features[:, feature])]
        classes_sorted = classes[np.argsort(features[:, feature])]
        sort_tracker.append((featured_sorted, classes_sorted))

        thresh = [(gini_gain(classes_sorted, [classes_sorted[:i+1], classes_sorted[i+1:]]), i, feature)
                  for i in range(0, len(features) - 1) if featured_sorted[i, feature] != featured_sorted[i+1, feature]]

        # split_point = features[find_thresh(featured_sorted[:,i], classes),i]
        
        # left_sub_y = features[:,i] < split_point
        # right_sub_y = features[:,i] > split_point

        # gini_list.append((round(gini_gain(classes_sorted, [classes_sorted[left_sub_y], classes_sorted[right_sub_y]]), 10), float(split_point), i))

val, split_point, best_alph = list(max(thresh))

curr_features = sort_tracker[best_alph][0]
curr_classes = sort_tracker[best_alph][1]


split_value = curr_features[split_point, best_alph]


In [ ]:
__build_tree__(data[0], data[1], depth= 0)

In [ ]:
tree = __build_tree__(data[0], data[1], depth= 0)
t = [tree.decide(i) for i in data[0]]

In [167]:
Counter([t[i] == data[1][i] for i in range(0,len(t))]).most_common()[0][1]/ len(t)

0.8454810495626822

In [139]:
Counter([t[i] == data[1][i] for i in range(0,len(t))]).most_common()[0][1]/ len(t)


0.8440233236151603

In [144]:
Counter([t[i] == data[1][i] for i in range(0,len(t))]).most_common()[0][1]/ len(t)

0.8454810495626822

In [107]:
__build_tree__(data[0], data[1], depth= 10).decide([-1.5681, -7.2446,  6.5537, -0.1276])

LEFT <----
LEFT <----


1.0

In [117]:
def jf(features, classes, depth=0) :
    class_count = Counter(classes)
    if depth == depth_limit or len(class_count) == 1:
        return DecisionNode(None, None, None, class_count.most_common()[0][0])
    
    #find best feature-threshold combo
    best_gain = 0
    best_f_idx = -1
    best_t_idx = -1
    best_sorted_features = None
    best_sorted_classes = None
    for f_idx in range(features.shape[1]):

        #sort dataset by feature values (won't change distribution or count of values, maintains sync)
        sorted_idxs = np.argsort(features[:, f_idx])
        sorted_features = features[sorted_idxs]
        sorted_classes = classes[sorted_idxs]

        #check all existing feature values as thresholds
        for t_idx in range(features.shape[0] - 1):

            #skip duplicate thresholds
            if sorted_features[t_idx+1][f_idx] == sorted_features[t_idx][f_idx]:
                continue

            #find gini gain at current threshold
            d1 = sorted_classes[:t_idx+1]
            d2 = sorted_classes[t_idx+1:]
            curr_gain = gini_gain(sorted_classes, [d1, d2])

            #update alpha_best
            if curr_gain > best_gain:
                best_gain = curr_gain
                best_f_idx = f_idx
                best_t_idx = t_idx
                best_sorted_features = sorted_features
                best_sorted_classes = sorted_classes
    
    #return most likely label as no further information can be gained
    if best_gain == 0:
        return DecisionNode(None, None, None, class_count.most_common()[0][0])
    
    #build tree and recurse
    threshold = best_sorted_features[best_t_idx][best_f_idx]
    left = __build_tree__(best_sorted_features[:best_t_idx+1], best_sorted_classes[:best_t_idx+1], depth+1)
    right = __build_tree__(best_sorted_features[best_t_idx+1:], best_sorted_classes[best_t_idx+1:], depth+1)
    return DecisionNode(left, right, lambda in_features: in_features[best_f_idx] <= threshold)

In [ ]:
jt = [jf(data[0], data[1], depth= 10).decide(i) for i in data[0]]

In [181]:
features = data[0]
classes = data[1]
thresh = [sorted(features[:,0])[i] + sorted(features[:,0])[i+1] / 2 for i in range(0, len(features[:,0]) - 1, 1)]
gini_line = []
for split_point in thresh:
    left_sub_y = features[:,0] < split_point
    right_sub_y = features[:,0] > split_point
    gini_line.append((round(gini_gain(classes, [classes[left_sub_y], classes[right_sub_y]]), 10), split_point))
    
np.where(thresh == max(gini_line)[1])


(array([638], dtype=int64),)

In [180]:
gini_line = []
for split_point in thresh:
    left_sub_y = features[:,0] < split_point
    right_sub_y = features[:,0] > split_point
    gini_line.append((round(gini_gain(classes, [classes[left_sub_y], classes[right_sub_y]]), 10), split_point))

np.where(thresh == max(gini_line)[1])


(array([638], dtype=int64),)

In [6]:
data[1]

array([30., 40., 44., ..., 68., 59., 65.])

In [ ]:
class DecisionTree:
    """Class for automatic tree-building and classification."""

    def __init__(self, depth_limit=float('inf')):
        """Create a decision tree with a set depth limit.
        Starts with an empty root.
        Args:
            depth_limit (float): The maximum depth to build the tree.
        """

        self.root = None
        self.depth_limit = depth_limit

    def fit(self, features, classes):
        """Build the tree from root using __build_tree__().
        Args:
            features (m x n): m examples with n features.
            classes (m x 1): Array of Classes.
        """

        self.root = self.__build_tree__(features, classes)

    
    def __build_tree__(self, features, classes, depth=0):
        """Build tree that automatically finds the decision functions.
        Args:
            features (m x n): m examples with n features.
            classes (m x 1): Array of Classes.
            depth (int): depth to build tree to.
        Returns:
            Root node of decision tree.
        """

        if depth == self.depth_limit :
            y = Counter(classes).most_common()[0][0]
            return DecisionNode(None, None, None, y)
    
        if len(classes) == 1 :
            y = Counter(classes).most_common()[0][0]
            return DecisionNode(None, None, None, y)
        
        # Finding the best split
        ledger = [0,0,0,None,None]
        for feature in range(features.shape[1]):

            featured_sorted = features[np.argsort(features[:, feature])]
            classes_sorted = classes[np.argsort(features[:, feature])]

            for dp in range(0, len(features) - 1):

                curr_gini_info = gini_gain(classes_sorted, [classes_sorted[:dp+1], classes_sorted[dp+1:]])

                if curr_gini_info > ledger[0]:
                    ledger[0] = curr_gini_info
                    ledger[1] = feature
                    ledger[2] = dp
                    ledger[3] = classes_sorted
                    ledger[4] = featured_sorted

        if ledger[0] <= 0:
            y = Counter(classes).most_common()[0][0]
            return DecisionNode(None, None, None, y)

        curr_features = ledger[4]
        curr_classes = ledger[3]

        
        split_value = curr_features[ledger[2], ledger[1]]
        left_leaf = self.__build_tree__(curr_features[:ledger[2]+1], curr_classes[:ledger[2]+1], depth = depth + 1)
        right_leaf = self.__build_tree__(curr_features[ledger[2]+1:], curr_classes[ledger[2]+1:], depth = depth + 1)

        curr_root = DecisionNode(left_leaf, right_leaf, lambda feat: feat[ledger[1]] <= split_value)

        return curr_root


    def classify(self, features):
        """Use the fitted tree to classify a list of example features.
        Args:
            features (m x n): m examples with n features.
        Return:
            A list of class labels.
        """
        class_labels = [self.root.decide(feature) for feature in features]
        # class_labels = self.root.decide(feature=features)
        return class_labels
   

class RandomForest:
    """Random forest classification."""

    def __init__(self, num_trees, depth_limit, example_subsample_rate,
                 attr_subsample_rate):
        """Create a random forest.
         Args:
             num_trees (int): fixed number of trees.
             depth_limit (int): max depth limit of tree.
             example_subsample_rate (float): percentage of example samples.
             attr_subsample_rate (float): percentage of attribute samples.
        """

        self.trees = []
        self.num_trees = num_trees
        self.depth_limit = depth_limit
        self.example_subsample_rate = example_subsample_rate
        self.attr_subsample_rate = attr_subsample_rate

    def fit(self, features, classes):
        """Build a random forest of decision trees using Bootstrap Aggregation.
            features (m x n): m examples with n features.
            classes (m x 1): Array of Classes.
        """
        for tree in range(self.num_trees):
            dp_sample_indx = np.random.choice(features.shape[0], round(self.example_subsample_rate * features.shape[0]), replace=True)
            feature_samples = features[dp_sample_indx]
            class_samples = classes[dp_sample_indx]

            dt = DecisionTree(depth_limit = self.depth_limit)
            dt.fit(feature_samples, class_samples)
            self.trees.append(dt)

    def classify(self, features):
        """Classify a list of features based on the trained random forest.
        Args:
            features (m x n): m examples with n features.
        """

        ballet = np.array([dt.classify(features) for dt in self.trees])
        return([Counter(ballet[:,label]).most_common()[0][0] for label in range(features.shape[0])])
